In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict

import keras
import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

np.random.seed(7)

In [2]:
df=pd.read_csv('open/train.csv', encoding='utf-8')
a2c = {0: 0, 1 : 1, 2 : 2, 3 : 3, 4 : 4}
y = np.array([a2c[a] for a in df.author])
y = to_categorical(y)

In [3]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [4]:
def preprocess(text):
    text = text.replace("' ", " ' ")
    signs = set('\"\'“”‘’,.:;?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

In [5]:
def create_docs(df, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocess(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

docs = create_docs(df)
docs

{',', ';', '.'}
{'?', '”', ',', '“'}
{'’', ',', '.', ':'}
{',', ';', '.', '”', "'", '“'}
{',', ';', '.', '”', '“', '!', '’'}
{'"', ',', '.'}
{',', '.'}
{'”', ',', '“', '.'}
{',', "'", '.'}
{',', '.', '”', '“', '!'}
{'“', '.'}
{'”', ',', '“', '.'}
{',', '.'}
{',', '.'}
{'?', '”', '“', '.'}
{',', '.'}
{',', '.'}
{'”', '“', '.', '’'}
{'?', ',', ';', '.'}
{',', '.', '’', '‘'}
{',', ';', '.'}
{'?', '”', '“', '!', '’'}
{',', '.'}
{',', '.'}
{',', '.', '”', '“', '!'}
{','}
{'”', ',', '“', '.'}
{',', ';', '.', '”', "'", '“'}
{',', '.'}
{',', '.', '”', '‘', '“', '’'}
{'"', ',', "'", '.'}
{'?', "'", '.'}
{',', ';', '.'}
{',', '.'}
{',', '.', '”', '“', '!'}
{'?', ',', "'", '.'}
{'”', ',', '“', '.'}
{',', '.'}
{',', '.', '?', '‘', '’'}
{'”', ',', '“', '.'}
{'”', '“', '!'}
{'"', ',', '.'}
{'?', '"', ','}
{'”', ',', '“', '.'}
{'”', ',', '“', '.'}
{'”', ',', '“', '.'}
{',', '.', '’'}
{',', '.', '?', '”', '“', '’'}
{',', ';', '.', '”', '“', '’'}
{'”', ',', '“', '.'}
{'”', ',', '“', '.'}
{',', '?', '”'

{',', ';', '.', '?', '"', '!', ':'}
{',', '.', '‘', '!', '’'}
{',', ';', '.', '”', '“'}
{',', '.', '’'}
{',', '.', '”', '“', '!', '’'}
{',', ';', '.', '?', '”', '“', '!', '’'}
{'”', ',', '“', '.'}
{'”', '“', '.'}
{',', ';', '.'}
{'”', "'", '.', '“'}
{'"', ',', '.'}
{',', '.'}
{'”', ',', '“', '.'}
{',', '.', '?', '”', '“'}
{',', '.'}
{',', '.', '?', '"', "'"}
{',', '.', '”', "'", '“'}
{',', '.'}
{'”', '!', '“', '.'}
{'?', '"', ',', '.'}
{',', ';', '.', '?', '"', "'"}
{'”', '“', '.'}
{'.', '”', "'", '!', '“'}
{'?', '"', ',', "'"}
{',', '.', '”', "'", '!', '“'}
{',', '.', '?', '”', '“'}
{',', '.', '?', '”', '‘', '“', '!'}
{',', ';', '.', '”', '“'}
{'.'}
{',', '.', '’'}
{',', '.', '?', '‘', '’'}
{'"', ';', '.', "'"}
{'”', ',', '“', '.'}
{',', '.', '?', '”', '“'}
{'.', '?', '‘', '“', '’'}
{',', '.', '?', '”', '“'}
{',', '.', ':'}
{'?', '”', '“', '’'}
{'”', '“', '.', '’'}
{'?', '”', '“'}
{',', '.', '”', '“', '!'}
{',', '.', '”', '“', '’'}
{',', '.', '”', '“', '!', '’'}
{',', '.', '”', '“', '

{',', ';', '.'}
{'”', '!', '“', '.'}
{'?', '"', ',', '.'}
{',', ';', '.'}
{'.'}
{',', '.', '”', '“', '’'}
{',', '“', '.', '’'}
{',', '.', '’', '‘'}
{'"', '!', ',', '.'}
{'”', ',', '“', '.'}
{'?', '”', ',', '“'}
{'”', ';', '.', '“'}
{',', ';', '.', "'"}
{',', '.', ':'}
{'?', '”', '“', '.'}
{',', '.'}
{',', '.', '”', '“', '!', '’'}
{'”', '“'}
{',', ';', '.'}
{',', '.'}
{'!', ';', '.', "'"}
{',', ';', '.', '‘', '’'}
{'.'}
{',', '.', '?', '‘', '’'}
{'”', ',', '“', '.'}
{'?', '”', '“', '.'}
{'"', ',', ';', '.'}
{',', '.', '?', '”', '“'}
{',', '.', '?', '”', '“', '!', '’'}
{'?', '”', ',', '“'}
{',', '.'}
{',', '.', '”', '“', '!'}
{',', ';', '.', '"', '!'}
{',', '.', '?', '”', '“'}
{';', '.', '”', '“', '!'}
{',', '.', '”', '“', '’'}
{',', ';', '.', '?', '”', '“', '!'}
{'!', ',', "'", '.'}
{'?', '"', '.'}
{',', '.', '?', '‘', '’'}
{',', ';', '.', '?', '”', '“', '!', '’'}
{',', '.'}
{',', ';', '.', '”', "'", ':', '“'}
{',', ';', '.', "'"}
{',', '.', '’'}
{',', '.'}
{',', '.', '”', '“', '’'}
{'.

{',', '.', '?', '”', "'", '!', '“'}
{',', '.', '”', '“', '’'}
{',', '.'}
{',', ';', '.', '?', '”', '“', '!', '’'}
{'.'}
{'”', ',', '“', '.'}
{',', '.', '”', '“', '’'}
{',', ';', '.'}
{'?', "'", '.'}
{',', '.', '‘', '“', '’'}
{',', '.', '’', '‘'}
{',', '.', '”', '“', '’'}
{',', '.', '?', '”', '“'}
{',', '.'}
{',', '.', '?', '‘', '’'}
{'?', '”', '“', '.'}
{'?', '”', ',', '“'}
{',', '.'}
{'”', ',', '“', '.'}
{',', ';', '.', "'"}
{'.'}
{'?', ',', '’'}
{',', ';', '.'}
{'?', '”', ',', '“'}
{',', '.', '?', '”', '“'}
{',', ';', '.', '!', '’'}
{',', ';', '.', '”', '“'}
{',', ';', '.'}
{',', '.'}
{'?', ',', "'", '.'}
{',', ';', '.', '”', '“'}
{',', "'", '.'}
{',', '.', '”', '“', '’'}
{',', '.', '’'}
{'?', "'", '.'}
{',', '”', '“', '!', '’'}
{',', ';', '.', '’'}
{'"', ',', "'", '.'}
{'"', ',', '.'}
{',', '.'}
{'”', ',', '“', '.'}
{',', ';', '?', '”', '“', '’'}
{'.', '?', '‘', '“', '’'}
{',', '.', '”', '“', '!'}
{'?', ',', '’', '‘'}
{',', '.', '?', '”', '“'}
{',', '.', '”', '“', '’'}
{',', '.', '”

{',', '.', '’'}
{'”', ',', '“', '.'}
{',', '.', '?', '”', "'", '“'}
{',', '.', '”', '“', '’'}
{'!', ',', "'", '.'}
{'?', ',', "'", '.'}
{',', ';', '.', '!', '’'}
{'?', '"', '.'}
{',', '.', '?', '"', "'", '!', ':'}
{',', '.', '?', '"', "'"}
{',', ';', '.', '’'}
{',', ';', '.', "'"}
{',', '.', '?', '‘', '“', '’'}
{'?', '”', '“', '’'}
{',', '.', '”', '“', '’'}
{',', '.', '?', '”', '“', '’'}
{',', '.', '?', '”', '“', '!', '’'}
{'”', ',', '“', '.'}
{',', '.'}
{'?', '"', ','}
{'"', ',', '.'}
{',', ';', '.', '"', "'"}
{'’', ',', '.', '?', '”', '‘', '!', ':'}
{',', '.', '‘', '!', '’'}
{',', '.', '’', '‘'}
{'"', '.'}
{'.'}
{'.'}
{',', '.'}
{',', '.', '?', '”', '“'}
{','}
{',', '.', '?', '”', '“', '!'}
{'?', ',', '.', '’'}
{',', '.', '?', '”', '“', '!', '’'}
{',', '.'}
{',', '.', '?', '‘', '’'}
{',', ';', '.', '”', '“', '’'}
{'"', '!', ',', '.'}
{'”', ',', '“', '.'}
{',', '.', '”', '“', '!', '’'}
{',', ';', '.'}
{',', '.', '?', '”', '“', '’'}
{'”', ',', '“', '.'}
{',', '.', '‘', '!', '’'}
{',', 

...

{'”', ';', '!', '“'}
{',', ';', '.'}
{',', ';', '.'}
{'.', '”', "'", '!', '“'}
{',', ';', '.', '”', "'", '“'}
{',', '.', '?', '”', '“', '’'}
{'”', ',', '“', '.'}
{',', ';', '.', '”', '“', '’'}
{'"', ',', "'", '.'}
{'?', '”', '“'}
{',', ';', '.', ':'}
{',', '.', '?', '”', '“'}
{'”', ',', '“', '.'}
{',', "'", '.'}
{'”', ',', '“', '.'}
{'”', ',', '“', '.'}
{',', '.', '?', '!', '’'}
{'.', '”', '“', '!', '’'}
{',', '.', '?', '”', '“', '!', '’'}
{'"', ',', '.'}
{',', ';', '.', '‘', '!', '’'}
{',', ';', '?', '‘', '’'}
{'"', ',', "'", '.'}
{',', ';', '.'}
{'”', ',', '“', '.'}
{'”', ',', '“', '.'}
{'?', '"'}
{'"', ',', "'", '!'}
{',', '.'}
{'.'}
{',', ';', '.', '"', "'"}
{',', '.', '?', '”', '“'}
{'"', '!', ',', '.'}
{'?', ',', ';', '.'}
{',', '.', '’'}
{'?', '”', '“', '’'}
{',', '.', '‘', '!', '’'}
{'"', ',', "'", '.'}
{',', '.', '?', '”', '“'}
{',', ';', '.', "'", ':'}
{',', ';', '.', '”', '“', '’'}
{',', '.'}
{',', ';', '.', '”', '“', '’'}
{',', '.'}
{'?', '"', "'"}
{',', '.', '?', '”', '“',

{',', ';', '.', '?', '”', '‘', '“', '’'}
{',', '.'}
{'!', ',', ';', '.'}
{',', ';', '.', '’'}
{',', '.', '?', '”', '“'}
{',', '.'}
{'?', '"'}
{',', "'", '.'}
{',', '.', '?', '"', "'"}
{'”', '“', '.'}
{'"', "'", '.'}
{',', '.', '?', '”', '“'}
{',', '.'}
{',', '.', '’'}
{',', '?', '”', '‘', '“', '’'}
{',', '.'}
{',', ';', '.', '?', '”', '“', '’'}
{',', ';', '.'}
{',', '.', '’', '‘'}
{'"', '.'}
{',', ';', '.', '?', "'", '!'}
{'"', ',', "'", '.'}
{',', ';', '.', '”', "'", '!', '“'}
{',', '.', '”', '“', '!'}
{',', "'"}
{'?', '”', '“'}
{',', '.'}
{',', '.', '”', '“', '’'}
{'.'}
{',', ';', '.', "'"}
{',', '.', '”', "'", '“'}
{',', '.', '?', '”', '“'}
{'"', ',', ';', '.'}
{',', ';', '.', '”', '“', '’'}
{'?', '”', ',', '“'}
{',', '.', '?', '”', '“', '!'}
{',', '.', '’'}
{',', ';', '.', '”', '“', '’'}
{',', "'", '.', ':'}
{'?', '”', '“', '.'}
{',', '.'}
{',', ';', '.'}
{',', ';', '.', '’'}
{',', '.', '”', '“', '!', '’'}
{'?', '"', ','}
{',', ';', '.', '”', '“'}
{'?', '”', ',', '“'}
{'"', '!', ',

{'”', ',', '“', '.'}
{',', '.', '’'}
{',', '.', '’', '‘'}
{',', '.', '’', '‘'}
{',', '.', '?', '‘', '’'}
{'!', ',', ';', '.'}
{',', '!', '’', '‘'}
{'"', ',', '.'}
{',', '.', '?', '”', '“', '!', '’'}
{',', '.', '’', '‘'}
{',', '.'}
{',', '.', '”', '“', '’'}
{',', "'", '.'}
{',', "'", '.'}
{'”', ',', '“', '.'}
{',', ';', '.', '”', '“', '!', '’'}
{',', ';', '.'}
{',', ';', '.', '?', '”', '“', '!'}
{',', "'", '.'}
{',', "'", '.', ':'}
{',', '.'}
{'?', '”', '“', '.'}
{'”', ',', '“', '.'}
{'”', ',', '“', '.'}
{',', ';', '.'}
{'”', ',', '“', '.'}
{'.', '?', '”', '“', '!'}
{',', '?', '”', '“', '!'}
{',', ';', '.'}
{'”', ',', '“', '.'}
{',', '.', '”', '“', '’'}
{'"', ',', '!'}
{',', '.', '?', '”', '“', '!'}
{'!', ',', ';', '.'}
{',', ';', '.', '”', '“'}
{',', ';', '.', '"', "'"}
{'?', '”', '“'}
{',', '.', '?', '”', '“'}
{'”', '“', '.'}
{',', '.', '”', '“', '’'}
{'"', '!'}
{'.'}
{',', '.', '’', '‘'}
{'?', '”', '“', '.'}
{',', '.', '?', '”', '“', '’'}
{',', ';', '.', '"', "'"}
{'?', "'", '.'}
{',

{',', '.', '"', "'", '!'}
{',', '.', '”', '“', '!'}
{'?', '”', ',', '“'}
{',', ';', '.', '"', ':'}
{',', ';', '.', '?', '”', "'", '“'}
{'!', '.', '’', '‘'}
{',', '.'}
{',', '.', '?', '”', "'", '“'}
{'”', ',', '“', '.'}
{',', ';', '.', '?', '”', '“'}
{'?', '”', '“', '.'}
{',', ';', '.', '”', '“', '’'}
{'”', '!', '“', '.'}
{',', '.', '’', '‘'}
{'?', '”', '“'}
{',', "'", '.'}
{',', '.', '”', '“', '’'}
{'"', ',', '!'}
{',', ';', '.', '’'}
{'”', ',', '“', '.'}
{',', '.', '‘', '!', '’'}
{',', '.', '‘', '!', '’'}
{',', '.', '?', '”', '“'}
{',', ';', '.', '?', '”', '“', '’'}
{',', '.', '?', "'", ':'}
{',', '.'}
{',', ';', '.', '‘', '’'}
{',', '.'}
{'”', '“', '.'}
{',', '?', '”', '“', '!'}
{',', '.', '‘', '!', '’'}
{'"', ',', "'", '.'}
{',', '.', '’', '‘'}
{'”', '“', '.'}
{',', '?', '”', '“', '!'}
{',', ';', '.', '"', "'", '!'}
{'"', '!', ',', '.'}
{',', ';', '.', '?', '”', '‘', '“', '!', '’'}
{'”', ',', '“', '.'}
{',', '.'}
{',', ';', '.', '?', '"', "'", '!'}
{',', '.'}
{'?', '”', '“', '’'}
{'

['He was almost choking . There was so much , so much he wanted to say , but strange exclamations were all that came from his lips . The Pole gazed fixedly at him , at the bundle of notes in his hand ; looked at odin , and was in evident perplexity . He--was was--almost almost--choking choking--. .--There There--was was--so so--much much--, ,--so so--much much--he he--wanted wanted--to to--say say--, ,--but but--strange strange--exclamations exclamations--were were--all all--that that--came came--from from--his his--lips lips--. .--The The--Pole Pole--gazed gazed--fixedly fixedly--at at--him him--, ,--at at--the the--bundle bundle--of of--notes notes--in in--his his--hand hand--; ;--looked looked--at at--odin odin--, ,--and and--was was--in in--evident evident--perplexity perplexity--.',
 '“ Your sister asked for it , I suppose ? ” “--Your Your--sister sister--asked asked--for for--it it--, ,--I I--suppose suppose--? ?--”',
 'She was engaged one day as she walked , in perusing Jane ’ s

In [6]:
min_count = 2

tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(docs)

num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)

maxlen = max([max(len(l) for l in docs)])

docs = pad_sequences(sequences=docs, maxlen=maxlen)

In [7]:
input_dim = np.max(docs) + 1
embedding_dims = 64

In [8]:
def create_model(embedding_dims=64, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [9]:
# post 추가
# from tensorflow.python.types import core as core_tf_types

epochs = 25
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Epoch 1/25
1451/2744 [==============>...............] - ETA: 5:02 - loss: 1.5474 - accuracy: 0.3097

KeyboardInterrupt: 

In [ ]:
predictions = model.predict_proba(x_test)
print ("logloss: %0.3f " % multiclass_logloss(y_test, predictions))
score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)

print('테스트 정확도:', score[1])
print('테스트 loss:', score[0])

WARNING:tensorflow:From <ipython-input-11-4d477344873d>:1: Sequential.predict_proba (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use `model.predict()` instead.
logloss: 0.484 
테스트 정확도: 0.8386479616165161
테스트 loss: 0.48444488644599915

In [ ]:
test_df = pd.read_csv('open/test_x.csv')
sample_submission=pd.read_csv('open/sample_submission.csv', encoding='utf-8')
docs = create_docs(test_df)
docs = tokenizer.texts_to_sequences(docs)
docs = pad_sequences(sequences=docs, maxlen=maxlen)
pred = model.predict_proba(docs)
sample_submission[['0', '1', '2', '3', '4']] = pred
sample_submission.to_csv('fkc_submission_1123.csv', index=False, encoding='utf-8')

In [ ]:
print('Done')